In [2]:
#dls with goal based:
#Task1a
class GoalBasedAgentDLS:
    def __init__(self, goal, depth_limit):
        self.goal = goal
        self.depth_limit = depth_limit
        self.visited = []
    def formulate_goal(self, node, depth):
        if depth > self.depth_limit:
            return "Depth Limit Reached"
        if node == self.goal:
            return "Goal Reached"
        return "Continue Search"
    def act(self, graph, node, depth):
        self.visited.append(node)
        status = self.formulate_goal(node, depth)
        if status == "Goal Reached":
            print(f"Goal found with DLS. Path: {self.visited}")
            return True
        elif status == "Depth Limit Reached":
            return False
        for neighbor in graph.get(node, []):
            if neighbor not in self.visited:
                found = self.act(graph, neighbor, depth + 1)
                if found:
                    return True
        self.visited.pop()
        return False
class EnvironmentDLS:
    def __init__(self, graph, start):
        self.graph = graph
        self.start = start
    def get_percept(self):
        return self.start
def run_goal_based_agent(agent, environment):
    start = environment.get_percept()
    if not agent.act(environment.graph, start, 0):
        print("Goal not found within depth limit")
graph = {
    'A': ['B', 'C'],
    'B': ['D', 'E'],
    'C': ['F', 'G'],
    'D': ['H'],
    'E': [],
    'F': ['I'],
    'G': [],
    'H': [],
    'I': []
}

agent = GoalBasedAgentDLS(goal='D', depth_limit=3)
environment = EnvironmentDLS(graph, start='A')
run_goal_based_agent(agent, environment)


Goal found with DLS. Path: ['A', 'B', 'D']


In [3]:
#task1b
#ucs with utility based agent:
graph = {
    'A': {'B': 2, 'C': 1},
    'B': {'D': 4, 'E': 3},
    'C': {'F': 1, 'G': 5},
    'D': {'H': 2},
    'E': {},
    'F': {'I': 6},
    'G': {},
    'H': {},
    'I': {}
}
# Utility-Based Agent Class
class UtilityBasedAgent:
    def __init__(self, start, goal):
        self.start = start
        self.goal = goal
        self.frontier = [(start, 0)]
        self.visited = set()
        self.cost_so_far = {start: 0}
        self.came_from = {start: None}
    def calculate_utility(self, cost):
        return -cost
    def select_action(self, neighbors):
        for neighbor, cost in neighbors.items():
            new_cost = self.cost_so_far[self.current_node] + cost
            if neighbor not in self.cost_so_far or new_cost < self.cost_so_far[neighbor]:
                self.cost_so_far[neighbor] = new_cost
                self.came_from[neighbor] = self.current_node
                self.frontier.append((neighbor, new_cost))
    def act(self, environment):
        while self.frontier:
            self.frontier.sort(key=lambda x: x[1])
            self.current_node, current_cost = self.frontier.pop(0)
            if self.current_node in self.visited:
                continue
            self.visited.add(self.current_node)
            if self.current_node == self.goal:
                path = []
                while self.current_node is not None:
                    path.append(self.current_node)
                    self.current_node = self.came_from[self.current_node]
                path.reverse()
                print(f"Goal found with UCS. Path: {path}, Total Cost: {current_cost}")
                return
            neighbors = environment.get_percept(self.current_node)
            self.select_action(neighbors)
        print("Goal not found")
class Environment:
    def __init__(self, graph):
        self.graph = graph

    def get_percept(self, node):
        return self.graph.get(node, {})
def run_agent(agent, environment):
    agent.act(environment)
environment = Environment(graph)
agent = UtilityBasedAgent(start='A', goal='I')
run_agent(agent, environment)


Goal found with UCS. Path: ['A', 'C', 'F', 'I'], Total Cost: 8


In [4]:
#Task2:
from collections import deque
dist_matrix = [
    [0, 10, 15, 20],
    [10, 0, 35, 25],
    [15, 35, 0, 30],
    [20, 25, 30, 0]
]
cities = ['A', 'B', 'C', 'D']
num_cities = len(dist_matrix)
def ucs_tsp():
    queue = deque()
    queue.append((0, [0]))
    min_cost = float('inf')
    best_route = None
    while queue:
        cost, path = queue.popleft()
        if len(path) == num_cities:
            total_cost = cost + dist_matrix[path[-1]][path[0]]
            if total_cost < min_cost:
                min_cost = total_cost
                best_route = path + [path[0]]

        for next_city in range(num_cities):
            if next_city not in path:
                new_cost = cost + dist_matrix[path[-1]][next_city]
                new_path = path + [next_city]
                queue.append((new_cost, new_path))
    best_route_cities = [cities[i] for i in best_route]
    return best_route_cities, min_cost
best_route, min_cost = ucs_tsp()
print("Route:", best_route)
print("Distance:", min_cost)


Route: ['A', 'B', 'D', 'C', 'A']
Distance: 80


In [7]:
graph = {
    'A': ['B', 'C'],
    'B': ['D', 'E'],
    'C': ['F', 'G'],
    'D': [],
    'E': [],
    'F': [],
    'G': []
}
def dls(node, goal, depth, path):
    if depth == 0:
        return False
    if node == goal:
        path.append(node)
        return True
    if node not in graph:
        return False
    for child in graph[node]:
        if dls(child, goal, depth - 1, path):
            path.append(node)
            return True
    return False
def iddfs(start, goal, max_depth):
    for depth in range(max_depth + 1):
        print(f"Depth Limit: {depth}")
        path = []
        if dls(start, goal, depth, path):
            print("Path:", " - ".join(reversed(path)))
            return
    print("Goal not found within depth limit.")
iddfs('A', 'F', 3)


Depth Limit: 0
Depth Limit: 1
Depth Limit: 2
Depth Limit: 3
Path: A - C - F


In [8]:
from collections import deque
grid = [
    ['S', 0, 1, 0],
    [0, 0, 1, 0],
    [1, 0, 0, 'G'],
    [0, 1, 0, 0]
]
directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]
def bfs(grid):
    rows, cols = len(grid), len(grid[0])
    queue = deque()
    visited = set()
    for i in range(rows):
        for j in range(cols):
            if grid[i][j] == 'S':
                start = (i, j)

    queue.append((start, [start]))
    visited.add(start)
    while queue:
        (x, y), path = queue.popleft()
        if grid[x][y] == 'G':
            return path
        for dx, dy in directions:
            nx, ny = x + dx, y + dy
            if 0 <= nx < rows and 0 <= ny < cols and (nx, ny) not in visited:
                if grid[nx][ny] != 1:
                    visited.add((nx, ny))
                    queue.append(((nx, ny), path + [(nx, ny)]))

    return "No Path Found"
path = bfs(grid)
print("Shortest Path:", path)


Shortest Path: [(0, 0), (1, 0), (1, 1), (2, 1), (2, 2), (2, 3)]
